In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install dreamsboard -U

### 介绍
使用langchain进行任务规划，构建子任务的会话场景资源，通过MCTS任务执行器，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


## 使用

### 构建任务

- 初始化任务引擎 StructuredTaskStepStoryboard传入需要的任务

- loader_task_step_iter_builder 构建任务的子任务，完成后SimpleTaskStepStore可获取子任务信息

- init_task_engine_dreams 初始化场景加载资源，对子任务进行规划，获取会话的资源信息

```

    os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
    os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
    os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
    os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
    os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
    os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST
```

- init_task_engine_storyboard_executor 构建会话场景执行器，初始化一个会话

- storyboard_code_gen_builder 构建会话场景执行器, 对会话存储进行加载，加载失败重新构建

- generate_step_answer 通过会话场景 获取任务的答案





```

In [3]:
   
from dreamsboard.engine.storage.task_step_store.types import BaseTaskStepStore
from dreamsboard.engine.task_engine_builder.core import TaskEngineBuilder
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore
from langchain_community.chat_models import ChatOpenAI
from dreamsboard.dreams.builder_task_step.base import StructuredTaskStepStoryboard
from dreamsboard.engine.utils import concat_dirs
from dreamsboard.engine.storage.task_step_store.types import DEFAULT_PERSIST_FNAME
from dreamsboard.common.try_parse_json_object import try_parse_json_object
from dreamsboard.engine.memory.mctsr.prompt import RefineResponse
from dreamsboard.dreams.task_step_md.base import TaskStepMD
from dreamsboard.common import _get_assistants_tool

import logging
import threading
import os
from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 控制台打印
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

logger.addHandler(handler)

In [4]:

llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
kor_dreams_task_step_llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.95,
    top_p=0.70,
)

deepseek_llm = ChatOpenAI(
    openai_api_base=os.environ.get("DEEPSEEK_API_BASE"),
    model=os.environ.get("DEEPSEEK_API_MODEL"),
    openai_api_key=os.environ.get("DEEPSEEK_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
zhipuai_llm = ChatOpenAI(
    openai_api_base=os.environ.get("ZHIPUAI_API_BASE"),
    model=os.environ.get("ZHIPUAI_API_MODEL"),
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
if 'glm' in os.environ.get("API_MODEL"):

    tools= [ { "type": "web_search",   "web_search": {"enable": False ,"search_result": False   }}]
else:
    tools = []
llm_with_tools = llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )
kor_dreams_task_step_llm_with_tools = kor_dreams_task_step_llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )

from tests.test_builder_task_step.prompts import (
    AEMO_REPRESENTATION_PROMPT_TEMPLATE as AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST,
    STORY_BOARD_SCENE_TEMPLATE as STORY_BOARD_SCENE_TEMPLATE_TEST,
    STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE as STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST,
    EDREAMS_EVOLUTIONARY_TEMPLATE as EDREAMS_EVOLUTIONARY_TEMPLATE_TEST,
    EDREAMS_PERSONALITY_TEMPLATE as EDREAMS_PERSONALITY_TEMPLATE_TEST,
    DREAMS_GEN_TEMPLATE as DREAMS_GEN_TEMPLATE_TEST,
) 
os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST


# 存储
cross_encoder_path = "/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
embed_model_path = "/mnt/ceph/develop/jiawei/model_checkpoint/m3e-base"
start_task_context = "大模型中的LayerNorm和RMSNorm有什么区别？"
builder = StructuredTaskStepStoryboard.form_builder(
    llm_runable=llm_with_tools,
    kor_dreams_task_step_llm=kor_dreams_task_step_llm_with_tools,
    start_task_context=start_task_context,
    cross_encoder_path=cross_encoder_path,
    embed_model_path=embed_model_path
)

 

/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
    

In [5]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=True)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
d753c815-d196-4a48-b95c-c565926fb392
e408c292-33f5-459e-9724-9942a9d19d1d
cf852a98-8ebb-40f0-bf13-6ea29fc63dd5
4aa00861-de5d-4346-a6dd-80ca6b0e7aa7
e2b95384-9e4b-4445-94fd-17a46fdce3ea
db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd
30b2ef30-75d2-40c5-b100-d2fefb6cb330


In [5]:

from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./af417502216ec584b5cbf3a9fda1879e/storage")
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,d753c815-d196-4a48-b95c-c565926fb392,1,story_board0,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,分析近几年研究领域的技术框架与方法论,在大模型相关研究领域，归一化技术是重要的研究方向之一。LayerNorm和RMSNorm作为...,0,近几年在大模型相关研究领域，研究者从理论分析和实验对比等多方面对LayerNorm和RMSN...,"[{'ref_id': '454845924254196540', 'chunk_id': ...",随着研究的深入，研究人员还在探索如何将这些新的归一化策略与大模型中的其他技术如注意力机制、模...,
1,e408c292-33f5-459e-9724-9942a9d19d1d,2,story_board1,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,研究论文中采用的主要框架（如Transformer、GAN、BERT等）在不同任务中的应用与变体,在Transformer架构为主的众多模型中，LayerNorm和RMSNorm被广泛应用。...,1,在基于Transformer的计算机视觉模型中，LayerNorm和RMSNorm具体进行了...,"[{'ref_id': '454895409734360760', 'chunk_id': ...",在未来的研究中，LayerNorm和RMSNorm的改进方向可以集中在多模态数据处理和跨领域...,
2,cf852a98-8ebb-40f0-bf13-6ea29fc63dd5,3,story_board2,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,评估学术界的技术进步与局限性,- **技术进步**：LayerNorm和RMSNorm的出现显著提升了大模型训练的稳定性和...,2,LayerNorm 和 RMSNorm 在提升大模型训练稳定性和效率方面取得了显著进步，但在...,"[{'ref_id': '454984230824843304', 'chunk_id': ...",进一步地，针对归一化方法在不同任务中的表现，研究者们还探讨了其在强化学习和生成模型中的应用。...,
3,4aa00861-de5d-4346-a6dd-80ca6b0e7aa7,4,story_board3,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,探讨计算模型在不同数据集与应用场景下的适用性与泛化能力,- **适用性**：LayerNorm在多种自然语言处理数据集和任务中表现出较好的适用性，能...,3,在多领域、多模态数据场景下，为了提升LayerNorm和RMSNorm对不同类型数据融合的适...,"[{'ref_id': '454846008144214678', 'chunk_id': ...",在跨领域泛化的实验设计中，可以评估模型在从一个领域（如自然语言处理）迁移到另一个领域（如计算...,
4,e2b95384-9e4b-4445-94fd-17a46fdce3ea,5,story_board4,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,分析最新算法的稳定性与容错性,- **稳定性**：研究发现，LayerNorm通过对输入数据的归一化处理，在一定程度上提高...,4,LayerNorm 和 RMSNorm 在复杂动态环境及大规模数据下是如何具体优化以保持稳定...,"[{'ref_id': '454845924254196540', 'chunk_id': ...",在图像生成领域，LayerNorm和RMSNorm同样发挥着重要作用。图像数据具有丰富的纹理...,
5,db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd,6,story_board5,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,评估论文中提出的未来研究方向与挑战,- **未来研究方向**：许多论文提出探索更适用于多模态数据的归一化方法，以及如何进一步优化...,5,为了应对论文中提出的未来研究挑战，即设计通用归一化策略适应不同领域和模态数据，以及在提升模型...,"[{'ref_id': '454848781599813822', 'chunk_id': ...",分数/总分：8/10,
6,30b2ef30-75d2-40c5-b100-d2fefb6cb330,7,story_board6,大模型中的LayerNorm和RMSNorm有什么区别？,### 分析近几年研究领域的技术框架与方法论\n在大模型相关研究领域，归一化技术是重要的研究...,研究成果、方法的创新性与应用价值,- **研究成果**：明确了LayerNorm和RMSNorm在大模型中的性能表现、适用场景...,6,LayerNorm 和 RMSNorm 的研究成果为后续研究提供了哪些具体的重要参考？它们在...,"[{'ref_id': '454847819065993190', 'chunk_id': ...",随着技术的不断发展，LayerNorm和RMSNorm在更多新兴领域也开始崭露头角。在智能交...,




#### 场景加载模块设计方案
```

编写符合计算机科学领域的 故事情境提示词，生成研究情境（story_scenario_context），替换现有的langchain会话模板，
1、对这个提示词所要求的输入拆分成子任务，
2、对每个子任务指令转换为子问题，召回问题前3条，
3、对召回内容与问题拼接，合成会话内容变量（scene_content）


对每个子问题相关的召回内容，转换为第一人称的会话总结（研究场景（scene_monologue_context）），

1、对召回内容与问题拼接，对每个前3条组成一个总结任务的提示词，为每个任务标记唯一编号，组成任务上下文（story_board_summary_context）
2、加载编号和story_board_summary_context，转换会话信息
```



#### MCTS设计方案

```
MCT 自优化算法代表了蒙特卡洛树搜索（MCTS）与大型语言模型的结合，将不同的场景执行任务过程抽象为搜索树结构。树上的节点代表当前不同视角下的选择策略，而边表示主体对自身的反思。该算法的操作流程遵循 MCTS 算法的一般模式。

具体来说，我们采用模型的先验知识，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


1、对问题生成的子任务，生成一个合理的规划的节点
2、对每个节点创建一个MCTS任务，
3、输入 problem（总问题的子任务相关的子问题）
4、评分代码重构，将片段摘录器模块集成到一个关于_evaluate_answer逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于evaluate_system_prompt 的生成策略，具体的为编写一个关于带有评估的评估器，由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message

5、自我反思代码重构,将片段摘录器模块集成到一个关于self_refine逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于critic_system_prompt和refine_system_prompt的生成策略，critic_system_prompt为生成一个关于子问题相关的loader_cosplay_builder中自身不完美的评价内容，refine_system_prompt为不完美评价的思考过程和评分值。
具体的为编写一个关于带有评价的生成器和反思生成器，它们由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message
```



#### MCTS执行中的重要环节
```
MCTS中的约束规则如下，需要保证这些节点必须符合下面所定义的基本规则

提示约束：模型在奖励评分期间必须遵守最严格的标准。生成结果需要为JSON Response format
{
    "thought": "The thought process behind the answer.",
    "answer": "A float representing the answer to the problem."
}


高分抑制：评分节点中不存在满分反馈机制；任何超过 95 分的奖励都会按固定金额减少，以遏制过高分数。

重复采样：每次访问搜索树节点都涉及对节点奖励的重复采样，以增强自我评估的可靠性。需要注意的是，当对节点的子节点进行奖励采样时，我们也会对其父节点进行奖励采样，以增加奖励采样的样本量。
```



In [6]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

d753c815-d196-4a48-b95c-c565926fb392
e408c292-33f5-459e-9724-9942a9d19d1d
cf852a98-8ebb-40f0-bf13-6ea29fc63dd5
4aa00861-de5d-4346-a6dd-80ca6b0e7aa7
e2b95384-9e4b-4445-94fd-17a46fdce3ea
db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd
30b2ef30-75d2-40c5-b100-d2fefb6cb330


In [7]:
 


def worker(step: int, task_engine: TaskEngineBuilder, task_step_store: BaseTaskStepStore):
    owner = f"step:{step}, task_step_id:{task_engine.task_step_id}, thread {threading.get_native_id()}"
    logger.info(f"{owner}，任务开始")
    if not task_engine.check_engine_init():
        task_engine.init_task_engine()
        task_engine.init_task_engine_dreams()
        task_engine.init_task_engine_storyboard_executor()

    try:
        logger.info(f"{owner}，storyboard_code_gen_builder")
        code_gen_builder = task_engine.storyboard_code_gen_builder()
        task_step = task_engine.task_step_store.get_task_step(task_engine.task_step_id)
        if task_step.task_step_question_answer is None or len(task_step.task_step_question_answer) == 0:
            task_engine.generate_step_answer(code_gen_builder)
        
        logger.info(f"step:{step}, {owner}，get_mcts_node")
        mcts_node = task_engine.get_mcts_node()
        if step % 2 == 0:
            mcts_node.llm_runable = deepseek_llm
        if step % 3 == 0:
            mcts_node.llm_runable = zhipuai_llm
        logger.info(f"step:{step}, {owner}，get_mcts_node run")
        answer = mcts_node.run()
        
        mcts_node.print()
        print(answer)
        task_step.task_step_question_answer = answer 
        task_step_id = task_engine.task_step_id
        
        task_engine.task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage/{task_step_id}", basename=DEFAULT_PERSIST_FNAME)
        task_engine.task_step_store.persist(persist_path=task_step_store_path) 
        
        task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage", basename=DEFAULT_PERSIST_FNAME)
        task_step_store.persist(persist_path=task_step_store_path) 

    except Exception as e:
        logger.error("场景加载失败", e)

    logger.info(f"{owner}，任务结束")




if __name__ == "__main__":
    threads = []
    
    step =0
    task_step_store = builder.task_step_store
    # 初始化任务引擎
    task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
    while not task_engine_builder.empty():
       
        task_engine = task_engine_builder.get()
        step += 1
        t = threading.Thread(target=worker,
                             kwargs={"step": step, "task_engine": task_engine, "task_step_store": task_step_store},
                             daemon=True)
        t.start()
        threads.append(t) 
    
    for t in threads:
        t.join()

step:1, task_step_id:d753c815-d196-4a48-b95c-c565926fb392, thread 273878，任务开始
step:2, task_step_id:e408c292-33f5-459e-9724-9942a9d19d1d, thread 273879，任务开始
step:3, task_step_id:cf852a98-8ebb-40f0-bf13-6ea29fc63dd5, thread 273880，任务开始
step:4, task_step_id:4aa00861-de5d-4346-a6dd-80ca6b0e7aa7, thread 273881，任务开始
step:5, task_step_id:e2b95384-9e4b-4445-94fd-17a46fdce3ea, thread 273882，任务开始
step:6, task_step_id:db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd, thread 273883，任务开始
step:7, task_step_id:30b2ef30-75d2-40c5-b100-d2fefb6cb330, thread 273884，任务开始
step:1, task_step_id:d753c815-d196-4a48-b95c-c565926fb392, thread 273878，storyboard_code_gen_builder
step:4, task_step_id:4aa00861-de5d-4346-a6dd-80ca6b0e7aa7, thread 273881，storyboard_code_gen_builder
step:3, task_step_id:cf852a98-8ebb-40f0-bf13-6ea29fc63dd5, thread 273880，storyboard_code_gen_builder
step:5, task_step_id:e2b95384-9e4b-4445-94fd-17a46fdce3ea, thread 273882，storyboard_code_gen_builder
step:2, task_step_id:e408c292-33f5-459e-9724-9942a

Received: content='10' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 134, 'total_tokens': 137, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-25577797-c28b-4653-b15d-c9ae3240ae45-0'
Iteration complete for resource_evaluate_db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd.
Received: content='78' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 370, 'total_tokens': 373, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-plus', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-98e73278-ee67-419d-b5db-79f137bcf81b-0'
Iteration complete for resource_evaluate_cf852a98-8ebb-40f0-bf13-6ea29fc63dd5.


owner:event end thread 273880, _evaluate_answer
owner:register_event thread 273880
owner:getlock thread 273946, resource_id:resource_critic_db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd
owner:lock thread 273946, resource_id:resource_critic_db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd
owner:getlock thread 273952, resource_id:resource_critic_cf852a98-8ebb-40f0-bf13-6ea29fc63dd5
owner:lock thread 273952, resource_id:resource_critic_cf852a98-8ebb-40f0-bf13-6ea29fc63dd5
resource_evaluate_30b2ef30-75d2-40c5-b100-d2fefb6cb330: content='70' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 278, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4545cc95-8702-438c-8305-c65c2c413809-0'
owner:event end thread 273884, _evaluate_answer
owner:register_event thread 273884


Received: content='70' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 278, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4545cc95-8702-438c-8305-c65c2c413809-0'
Iteration complete for resource_evaluate_30b2ef30-75d2-40c5-b100-d2fefb6cb330.


owner:getlock thread 273961, resource_id:resource_critic_30b2ef30-75d2-40c5-b100-d2fefb6cb330
owner:lock thread 273961, resource_id:resource_critic_30b2ef30-75d2-40c5-b100-d2fefb6cb330
resource_evaluate_d753c815-d196-4a48-b95c-c565926fb392: content='60' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 253, 'total_tokens': 255, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-45b99a3c-1f35-4698-b418-b437a961b571-0'
resource_evaluate_e2b95384-9e4b-4445-94fd-17a46fdce3ea: content='30' response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 407, 'total_tokens': 409, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'ep-20241224194233-9mshp', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs

...

在进一步的研究中，可以关注归一化方法在不同硬件平台上的优化和部署问题。例如，针对GPU和TPU等高性能计算设备，设计专门的归一化算法，以充分利用硬件特性，提升计算效率。此外，归一化方法在联邦学习中的隐私保护机制也是一个重要研究方向。如何在保证数据隐私的前提下，实现高效的归一化处理，是一个亟待解决的问题。通过引入差分隐私技术，可以在不泄露敏感信息的情况下，进行有效的模型训练。

在跨领域应用方面，归一化方法的迁移学习能力也值得深入探讨。例如，在将自然语言处理领域的归一化策略迁移到计算机视觉任务时，如何自适应地调整参数，以适应不同领域的数据特征，是一个具有挑战性的问题。通过设计领域自适应的归一化机制，可以在减少领域间差异的同时，提升模型的泛化能力。

最后，归一化方法在实际应用中的可解释性和透明度也是一个重要研究方向。如何设计可解释的归一化策略，使研究人员和用户能够理解模型的决策过程，提升模型的信任度，是一个值得关注的问题。通过引入可解释性强的归一化方法，可以在保证模型性能的同时，提升其在实际应用中的接受度。


resource_refine_d753c815-d196-4a48-b95c-c565926fb392: content='### 优化后的回答:\n在大模型技术框架中，LayerNorm 和 RMSNorm 与多种主流架构有着紧密的关联与协同。以 Transformer 架构为例，LayerNorm 通常应用于多头注意力机制之后以及前馈神经网络之前，对输入进行归一化处理，有助于稳定模型训练，使得各层输入数据分布相对稳定，提升模型对不同特征的捕捉能力，进而增强模型整体性能。RMSNorm 同样可在类似位置发挥作用，其独特的计算方式能在一定程度上减少计算量，并且在某些场景下能更好地适应数据的动态范围，与其他组件协同优化模型性能。在 GAN 框架中，归一化操作对于生成器和判别器的训练稳定性至关重要。LayerNorm 和 RMSNorm 可以应用于网络的不同层，帮助平衡生成数据和真实数据的分布，防止模型在训练过程中出现梯度消失或爆炸等问题，促进生成器和判别器的对抗训练达到更好的平衡。在 BERT 模型中，这两种归一化方法有助于提升预训练模型对文本特征的学习能力，在不同的 Transformer 块中合理应用，能够优化模型对上下文信息的理解和表示，提高下游自然语言处理任务的性能。\n\n研究人员在理论分析 LayerNorm 和 RMSNorm 工作原理时，运用了多方面的数学理论和分析方法。从概率论角度，通过分析数据的统计特性，如均值、方差等，来理解归一化操作对数据分布的影响。例如，研究归一化后的数据是否更接近高斯分布，以及这种分布变化对模型后续计算的影响。在线性代数方面，利用矩阵运算和向量空间理论，分析归一化操作对特征向量和矩阵变换的作用，探究其如何改变数据在特征空间中的表示。在分析方法上，采用公式推导深入剖析归一化的计算过程，明确每个参数和操作对最终结果的贡献。同时，结合模型结构分析，研究归一化层在整个模型架构中的位置和连接方式，以及它如何与其他层相互作用，影响模型的整体行为。\n\n在实验对比方面，研究者精心设计实验设置。在数据集选择上，涵盖了不同规模和领域的数据集，如自然语言处理中的大规模文本数据集（如 Wikipedia 数据集）、计算机视觉中的图像分类数据集（如 ImageNet）等。模型架构方面，采用多种不同复杂度的模型，包括不同层数和隐藏层维度的 Tra

Received: content='### 优化后的回答:\n在大模型技术框架中，LayerNorm 和 RMSNorm 与多种主流架构有着紧密的关联与协同。以 Transformer 架构为例，LayerNorm 通常应用于多头注意力机制之后以及前馈神经网络之前，对输入进行归一化处理，有助于稳定模型训练，使得各层输入数据分布相对稳定，提升模型对不同特征的捕捉能力，进而增强模型整体性能。RMSNorm 同样可在类似位置发挥作用，其独特的计算方式能在一定程度上减少计算量，并且在某些场景下能更好地适应数据的动态范围，与其他组件协同优化模型性能。在 GAN 框架中，归一化操作对于生成器和判别器的训练稳定性至关重要。LayerNorm 和 RMSNorm 可以应用于网络的不同层，帮助平衡生成数据和真实数据的分布，防止模型在训练过程中出现梯度消失或爆炸等问题，促进生成器和判别器的对抗训练达到更好的平衡。在 BERT 模型中，这两种归一化方法有助于提升预训练模型对文本特征的学习能力，在不同的 Transformer 块中合理应用，能够优化模型对上下文信息的理解和表示，提高下游自然语言处理任务的性能。\n\n研究人员在理论分析 LayerNorm 和 RMSNorm 工作原理时，运用了多方面的数学理论和分析方法。从概率论角度，通过分析数据的统计特性，如均值、方差等，来理解归一化操作对数据分布的影响。例如，研究归一化后的数据是否更接近高斯分布，以及这种分布变化对模型后续计算的影响。在线性代数方面，利用矩阵运算和向量空间理论，分析归一化操作对特征向量和矩阵变换的作用，探究其如何改变数据在特征空间中的表示。在分析方法上，采用公式推导深入剖析归一化的计算过程，明确每个参数和操作对最终结果的贡献。同时，结合模型结构分析，研究归一化层在整个模型架构中的位置和连接方式，以及它如何与其他层相互作用，影响模型的整体行为。\n\n在实验对比方面，研究者精心设计实验设置。在数据集选择上，涵盖了不同规模和领域的数据集，如自然语言处理中的大规模文本数据集（如 Wikipedia 数据集）、计算机视觉中的图像分类数据集（如 ImageNet）等。模型架构方面，采用多种不同复杂度的模型，包括不同层数和隐藏层维度的 Transformer 模型、卷积神经网络（CNN）等。训练参数设置也经过细致调整，如学习率、

解析后的 JSON 对象: {'thought': '在大模型技术框架中，LayerNorm 和 RMSNorm 与多种主流架构有着紧密的关联与协同。以 Transformer 架构为例，LayerNorm 通常应用于多头注意力机制之后以及前馈神经网络之前，对输入进行归一化处理，有助于稳定模型训练，使得各层输入数据分布相对稳定，提升模型对不同特征的捕捉能力，进而增强模型整体性能。RMSNorm 同样可在类似位置发挥作用，其独特的计算方式能在一定程度上减少计算量，并且在某些场景下能更好地适应数据的动态范围，与其他组件协同优化模型性能。在 GAN 框架中，归一化操作对于生成器和判别器的训练稳定性至关重要。LayerNorm 和 RMSNorm 可以应用于网络的不同层，帮助平衡生成数据和真实数据的分布，防止模型在训练过程中出现梯度消失或爆炸等问题，促进生成器和判别器的对抗训练达到更好的平衡。在 BERT 模型中，这两种归一化方法有助于提升预训练模型对文本特征的学习能力，在不同的 Transformer 块中合理应用，能够优化模型对上下文信息的理解和表示，提高下游自然语言处理任务的性能。\n\n研究人员在理论分析 LayerNorm 和 RMSNorm 工作原理时，运用了多方面的数学理论和分析方法。从概率论角度，通过分析数据的统计特性，如均值、方差等，来理解归一化操作对数据分布的影响。例如，研究归一化后的数据是否更接近高斯分布，以及这种分布变化对模型后续计算的影响。在线性代数方面，利用矩阵运算和向量空间理论，分析归一化操作对特征向量和矩阵变换的作用，探究其如何改变数据在特征空间中的表示。在分析方法上，采用公式推导深入剖析归一化的计算过程，明确每个参数和操作对最终结果的贡献。同时，结合模型结构分析，研究归一化层在整个模型架构中的位置和连接方式，以及它如何与其他层相互作用，影响模型的整体行为。\n\n在实验对比方面，研究者精心设计实验设置。在数据集选择上，涵盖了不同规模和领域的数据集，如自然语言处理中的大规模文本数据集（如 Wikipedia 数据集）、计算机视觉中的图像分类数据集（如 ImageNet）等。模型架构方面，采用多种不同复杂度的模型，包括不同层数和隐藏层维度的 Transformer 模型、卷积神经网络（CNN）等。训练参数设置也经过细致调整，如学习率、批次大小等

随着对这些归一化方法研究的不断深入，研究人员开始尝试将 LayerNorm 和 RMSNorm 与其他新兴技术进行融合。例如，与强化学习相结合，通过智能调整归一化参数，使模型在动态环境中能够更好地适应数据变化。在强化学习的训练过程中，智能体可以根据环境反馈来动态调整归一化的参数，以优化模型的性能。这种融合方式为解决复杂动态任务提供了新的思路。同时，研究人员也在探索将其应用于量子计算领域。量子计算具有独特的计算特性，传统的归一化方法在量子计算环境中可能需要进行适应性调整。LayerNorm 和 RMSNorm 能否在量子计算中发挥作用，以及如何进行改进以适应量子计算的需求，成为了新的研究热点。


In [8]:

store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./af417502216ec584b5cbf3a9fda1879e/storage")
task_step_md = TaskStepMD(store_load)
md_text =   task_step_md.format_md() 
from IPython.display import Markdown, display

display(Markdown(md_text.text))


# 大模型中的LayerNorm和RMSNorm有什么区别？ 


### 分析近几年研究领域的技术框架与方法论 [task_id:0](d753c815-d196-4a48-b95c-c565926fb392)

随着对这些归一化方法研究的不断深入，研究人员开始尝试将 LayerNorm 和 RMSNorm 与其他新兴技术进行融合。例如，与强化学习相结合，通过智能调整归一化参数，使模型在动态环境中能够更好地适应数据变化。在强化学习的训练过程中，智能体可以根据环境反馈来动态调整归一化的参数，以优化模型的性能。这种融合方式为解决复杂动态任务提供了新的思路。同时，研究人员也在探索将其应用于量子计算领域。量子计算具有独特的计算特性，传统的归一化方法在量子计算环境中可能需要进行适应性调整。LayerNorm 和 RMSNorm 能否在量子计算中发挥作用，以及如何进行改进以适应量子计算的需求，成为了新的研究热点。

### 研究论文中采用的主要框架（如Transformer、GAN、BERT等）在不同任务中的应用与变体 [task_id:1](e408c292-33f5-459e-9724-9942a9d19d1d)

在自然语言处理领域，BERT模型通过LayerNorm在每一层的输出上进行归一化，显著提升了模型的训练稳定性和泛化能力。类似地，在计算机视觉领域，Vision Transformer（ViT）通过引入LayerNorm来适应图像数据的特性，从而在图像分类任务中取得了与卷积神经网络相媲美的性能。在生成对抗网络（GAN）中，RMSNorm被用于生成器的每一层，以改进生成图像的质量和多样性。在语音识别任务中，RMSNorm通过调整特征分布的均值和方差，显著提升了模型的识别准确率。在多模态数据处理方面，最新的研究表明，通过结合自适应归一化策略和注意力机制，可以在文本-图像联合建模任务中实现更好的特征对齐和融合。例如，CLIP模型通过结合LayerNorm和跨模态注意力机制，在图像-文本匹配任务中取得了显著的效果。未来的研究可以进一步探索如何设计一种通用的归一化策略，能够无缝适应不同领域和模态的数据，并解决其潜在的技术挑战。

### 评估学术界的技术进步与局限性 [task_id:2](cf852a98-8ebb-40f0-bf13-6ea29fc63dd5)

在具体的研究案例中，LayerNorm在BERT模型中的应用显著提升了其在GLUE benchmark上的表现，具体而言，平均得分提高了12%。而在生成对抗网络（GAN）中，RMSNorm的应用不仅提升了生成图像的质量，还在训练稳定性上表现出色，实验数据显示，训练过程中的模式崩溃现象减少了30%。然而，这些方法在面对多模态数据融合任务时，仍存在一定的局限性。例如，在文本-图像联合建模任务中，归一化方法未能有效处理不同模态数据的特征差异，导致模型性能未能达到预期。

此外，归一化方法在联邦学习中的应用也引起了广泛关注。在分布式数据环境下，LayerNorm和RMSNorm能够有效减少模型训练中的通信开销，提升模型的收敛速度。例如，在跨设备联邦学习场景中，使用LayerNorm的模型在保持隐私保护的同时，收敛速度提升了25%。然而，面对非独立同分布（Non-IID）数据时，这些方法的性能仍有待提升，具体在某医疗数据集上的实验表明，模型的诊断准确率下降了8%。

### 探讨计算模型在不同数据集与应用场景下的适用性与泛化能力 [task_id:3](4aa00861-de5d-4346-a6dd-80ca6b0e7aa7)

在跨领域泛化的实验设计中，可以评估模型在从一个领域（如自然语言处理）迁移到另一个领域（如计算机视觉）时的泛化能力。例如，可以设计一个实验，使用在自然语言处理任务上预训练的模型，在计算机视觉任务上进行微调，并分析归一化方法在跨领域泛化中的作用。此外，可以进一步探讨多模态数据泛化的挑战，如模态间的不一致性、数据分布差异等，并提出可能的解决方案，如模态对齐技术、跨模态注意力机制等。

### 分析最新算法的稳定性与容错性 [task_id:4](e2b95384-9e4b-4445-94fd-17a46fdce3ea)

在实际应用场景中，如自动驾驶领域，数据的稳定性和算法的容错性至关重要。传感器收集到的图像和雷达数据可能存在噪声、遮挡等问题。LayerNorm和RMSNorm在处理这些数据时，其稳定性和容错性直接影响自动驾驶模型的决策准确性。例如，在复杂路况下，车辆传感器可能会受到阳光反射、雨水干扰等影响，此时算法能否准确处理这些有噪声的数据，决定了车辆能否做出正确的行驶决策。

### 评估论文中提出的未来研究方向与挑战 [task_id:5](db3d6ad7-14d4-4d13-8ee3-b0b4f63647bd)

在进一步的研究中，可以关注归一化方法在不同硬件平台上的优化和部署问题。例如，针对GPU和TPU等高性能计算设备，设计专门的归一化算法，以充分利用硬件特性，提升计算效率。此外，归一化方法在联邦学习中的隐私保护机制也是一个重要研究方向。如何在保证数据隐私的前提下，实现高效的归一化处理，是一个亟待解决的问题。通过引入差分隐私技术，可以在不泄露敏感信息的情况下，进行有效的模型训练。

在跨领域应用方面，归一化方法的迁移学习能力也值得深入探讨。例如，在将自然语言处理领域的归一化策略迁移到计算机视觉任务时，如何自适应地调整参数，以适应不同领域的数据特征，是一个具有挑战性的问题。通过设计领域自适应的归一化机制，可以在减少领域间差异的同时，提升模型的泛化能力。

最后，归一化方法在实际应用中的可解释性和透明度也是一个重要研究方向。如何设计可解释的归一化策略，使研究人员和用户能够理解模型的决策过程，提升模型的信任度，是一个值得关注的问题。通过引入可解释性强的归一化方法，可以在保证模型性能的同时，提升其在实际应用中的接受度。

### 研究成果、方法的创新性与应用价值 [task_id:6](30b2ef30-75d2-40c5-b100-d2fefb6cb330)

在金融领域，市场数据瞬息万变且包含众多复杂因素，如股票价格波动、宏观经济指标变化等。LayerNorm可以对这些多维度的金融数据进行归一化，使得金融风险预测模型能够更精准地分析数据之间的关联，从而提高风险预测的准确性。例如在预测股票市场的系统性风险时，使用LayerNorm的模型能够更好地捕捉各种因素对风险的影响，为投资者提供更可靠的风险预警。RMSNorm在处理大规模金融交易数据时，通过快速归一化加速模型训练，让金融模型能够及时适应市场的动态变化，为高频交易策略的制定提供支持。


 



StructuredTaskStepStoryboard
    
对任务进行规划，生成段落之间组成一个动态上下文，  扩写任务步骤构建MCTS任务
    输入：
    	start_task_context： 起始任务
    	llm： 模型
    	kor_dreams_task_step_llm: 任务抽取模型
    	task_step_store: 任务存储器（SimpleTaskStepStore）
    	cross_encoder_path: ranking模型路径（sentence_transformers），当前设计耦合了业务，之后会单独设计一个召回模块

    任务：

        1、对任务（AEMO_REPRESENTATION_PROMPT_TEMPLATE）按照提示词要求进行扩写，将扩写任务步骤收集 （src/dreamsboard/dreamsboard/engine/entity/task_step、src/dreamsboard/tests/test_kor/test_kor3.py）

        2、收集每个任务后存储到磁盘（src/dreamsboard/dreamsboard/engine/storage/task_step_store）

        3、对每个子任务载入会话场景，然后按照扩写任务步骤构建，MCTS任务 loader_task_step_iter_builder


	》 TaskEngineBuilder 场景加载模块
		执行会话场景资源初始化，构建MCTS任务

    根据任务步骤，构建场景加载模块，生成资源文件csv
    根据每个任务，载入StructuredDreamsStoryboard 会话场景
    按照扩写任务步骤构建MCTS任务

		输入：
			task_step_id
			task_step_store: 任务存储器（SimpleTaskStepStore）
			start_task_context： 起始任务
			llm： 模型

		任务：
			init_task_engine：
				初始化任务引擎
        》TaskStepToQuestionChain 
        	输入：
        		client： 矢量库客户端
        		llm： 模型
					invoke_task_step_to_question：1、 对开始任务进行抽取，得到任务步骤，提示词所要求的输入拆分成子任务， 
					invoke_task_step_question_context： 2、对每个子任务指令转换为子问题，召回问题前3条，对任务步骤进行抽取，得到任务步骤的上下文
					export_csv_file_path: 3、对召回内容与问题 导出csv文件

			init_task_engine_dreams
				初始化场景加载资源
					StoryBoardDreamsGenerationChain
					对每个子任务通过职业提示词，载入会话场景
						1、构建场景信息（story_scenario_context），提示词（STORY_BOARD_SCENE_TEMPLATE）
						2、对任务上下文(story_board_summary_context)，构建第一人称数据(scene_monologue_context),提示词（STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE）
						3、对任务上下文(story_board_summary_context)，获取任务分析(evolutionary_step), 提示词（EDREAMS_EVOLUTIONARY_TEMPLATE）
						4、对任务分析(evolutionary_step)，分析对话预设信息（性格）， 提示词（EDREAMS_PERSONALITY_TEMPLATE）
						5、对任务上下文(story_board_summary_context)，场景信息story_scenario_context, 第一人称数据(scene_monologue_context)，
						生成关于人物职业的引导话术，提示词（DREAMS_GEN_TEMPLATE）

			init_task_engine_storyboard_executor

				构建会话场景执行器 StructuredDreamsStoryboard
					对剧本和分析结果进行结构化，将开放问题与性格分析结果进行结合。生成情景扮演会话场景执行器
			    此过程如下
			        对开放问题结果进行抽取，得到问题内容
			        对性格分析结果进行抽取，得到性格分析结果
			        增加系统提示词
			        根据剧本与任务性格基础情景扮演代码，根据每步的抽取析得到的问题，生成问题的答案
			       	在上下文中增加，关于人物职业的引导话术
	        		导出会话场景执行器
					
	    storyboard_code_gen_builder
	    	构建会话场景执行器

	    generate_step_answer
	    	获取主进程子任务的答案
	    
	    get_mcts_node
	    	构建MCTS树, 初始化当前任务相关的MCTS节点，并返回MCTS执行器
